In [1]:
import requests
import getpass
import socket 
import json
import zipfile
import io
import math
import os 
import shutil
import time
import matplotlib.pyplot as plt
import h5py
import re
# To read KML files with geopandas, we will need to enable KML support in fiona (disabled by default)
import pprint
from statistics import mean
from requests.auth import HTTPBasicAuth

## Get Token

In [2]:
uid = 'karans04'
email = 'karans_7@yahoo.co.in'
pswd = getpass.getpass('Earthdata Login password: ')

Earthdata Login password: ········


In [3]:
token_api_url = 'https://cmr.earthdata.nasa.gov/legacy-services/rest/tokens'
hostname = socket.gethostname()
ip = socket.gethostbyname(hostname)

data = {
    'token': {
        'username': uid,
        'password': pswd,
        'client_id': 'NSIDC_client_id',
        'user_ip_address': ip
    }
}

headers={'Accept': 'application/json'}
response = requests.post(token_api_url, json=data, headers=headers)
print(response)
token = json.loads(response.content)['token']['id']
print(token)

<Response [201]>
BE86221E-AE9C-BC75-9B27-9F19A536E63E


## Get Data

In [4]:
# Bounding Box spatial parameter in 'W,S,E,N' format

# Input bounding box
# Input lower left longitude in decimal degrees
LL_lon = '128.3700'
# Input lower left latitude in decimal degrees
LL_lat = '27.0000'
# Input upper right longitude in decimal degrees
UR_lon = '128.4848'
# Input upper right latitude in decimal degrees
UR_lat = '27.0900'

bounding_box = LL_lon + ',' + LL_lat + ',' + UR_lon + ',' + UR_lat

print(bounding_box)
#Bounding box subsetting (bbox) in same format as bounding_box
bbox = bounding_box

128.3700,27.0000,128.4848,27.0900


In [5]:
params = {
    'short_name': short_name,
    'version': latest_version,
    'page_size': 100,
    'page_num': 1,
    'bounding_box': bounding_box,
    'request_mode': 'async'
}

NameError: name 'short_name' is not defined

In [39]:
#Set NSIDC data access base URL
base_url = 'https://n5eil02u.ecs.nsidc.org/egi/request'


#Print API base URL + request parameters
API_request = f'{base_url}?short_name=ATL03&bounding_box={bounding_box}&token={token}&request_mode=async'

In [40]:
import requests
import bs4
r = requests.get(API_request)

In [45]:
soup= bs4.BeautifulSoup(r.text)
urls = []
files = soup.find_all('downloadurl')
for file in files:
    urls.append(file.text)

In [47]:
req_base='curl -O -J --dump-header out.txt '
req = req_base +urls[0]
req

'curl -O -J --dump-header out.txt ftp://n5eil02u.ecs.nasa.gov/DP7/ATLAS/ATL03.001/2018.10.21/ATL03_20181021062956_03460101_001_01.h5'

In [50]:
p=subprocess.Popen(req, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
text=p.stdout.read()
retcode=p.wait()

In [49]:
import subprocess

In [5]:
short_name = 'ATL03'
params = {
    'short_name': short_name
}

cmr_collections_url = 'https://cmr.earthdata.nasa.gov/search/collections.json'
response = requests.get(cmr_collections_url, params=params)
results = json.loads(response.content)
pprint.pprint(results)

{'feed': {'entry': [{'archive_center': 'NASA NSIDC DAAC',
                     'associations': {'services': ['S1568899363-NSIDC_ECS',
                                                   'S1613669681-NSIDC_ECS',
                                                   'S1613689509-NSIDC_ECS']},
                     'boxes': ['-90 -180 90 180'],
                     'browse_flag': False,
                     'coordinate_system': 'CARTESIAN',
                     'data_center': 'NSIDC_ECS',
                     'dataset_id': 'ATLAS/ICESat-2 L2A Global Geolocated '
                                   'Photon Data V001',
                     'has_formats': True,
                     'has_spatial_subsetting': True,
                     'has_temporal_subsetting': True,
                     'has_transforms': False,
                     'has_variables': True,
                     'id': 'C1511845988-NSIDC_ECS',
                     'links': [{'href': 'https://n5eil01u.ecs.nsidc.org/ATLAS/ATL03.001/',
  

In [6]:
# Find all instances of 'version_id' in metadata and print most recent version number

versions = [i['version_id'] for i in results['feed']['entry']]
latest_version = max(versions)
print(latest_version)

001


128.3700,27.0000,128.4848,27.0900


In [104]:
granule_search_url = 'https://cmr.earthdata.nasa.gov/search/granules'

granules = []
while True:
    response = requests.get(granule_search_url, params=params, headers=headers)
    results = json.loads(response.content)
    print(response)
    if len(results['feed']['entry']) == 0:
        # Out of results, so break out of loop
        break

    # Collect results and increment page_num
    granules.extend(results['feed']['entry'])
    params['page_num'] += 1

    
# Get number of granules over my area and time of interest
len(granules)

<Response [400]>


KeyError: 'feed'

In [105]:
granule_sizes = [float(granule['granule_size']) for granule in granules]

# Average size of granules in MB
mean(granule_sizes)

StatisticsError: mean requires at least one data point

In [30]:
# Total volume in MB
sum(granule_sizes)

8305.2576446533

In [13]:

# Query service capability URL 

from xml.etree import ElementTree as ET

capability_url = f'https://n5eil02u.ecs.nsidc.org/egi/capabilities/{short_name}.{latest_version}.xml'

print(capability_url)

https://n5eil02u.ecs.nsidc.org/egi/capabilities/ATL03.001.xml


In [14]:

# Create session to store cookie and pass credentials to capabilities url

session = requests.session()
s = session.get(capability_url)
response = session.get(s.url,auth=(uid,pswd))

root = ET.fromstring(response.content)

In [15]:
# collect lists with each service option

subagent = [subset_agent.attrib for subset_agent in root.iter('SubsetAgent')]

# variable subsetting
variables = [SubsetVariable.attrib for SubsetVariable in root.iter('SubsetVariable')]  
variables_raw = [variables[i]['value'] for i in range(len(variables))]
variables_join = [''.join(('/',v)) if v.startswith('/') == False else v for v in variables_raw] 
variable_vals = [v.replace(':', '/') for v in variables_join]

# reformatting
formats = [Format.attrib for Format in root.iter('Format')]
format_vals = [formats[i]['value'] for i in range(len(formats))]
format_vals.remove('')

# reprojection only applicable on ICESat-2 L3B products, yet to be available. 

# reformatting options that support reprojection
normalproj = [Projections.attrib for Projections in root.iter('Projections')]
normalproj_vals = []
normalproj_vals.append(normalproj[0]['normalProj'])
format_proj = normalproj_vals[0].split(',')
format_proj.remove('')
format_proj.append('No reformatting')

#reprojection options
projections = [Projection.attrib for Projection in root.iter('Projection')]
proj_vals = []
for i in range(len(projections)):
    if (projections[i]['value']) != 'NO_CHANGE' :
        proj_vals.append(projections[i]['value'])
        
# reformatting options that do not support reprojection
no_proj = [i for i in format_vals if i not in format_proj]

In [16]:
print(subagent)
if len(subagent) < 1 :
    agent = 'NO'

[{'id': 'ICESAT2', 'spatialSubsetting': 'true', 'spatialSubsettingShapefile': 'true', 'temporalSubsetting': 'true', 'type': 'both', 'maxGransSyncRequest': '100', 'maxGransAsyncRequest': '2000'}]


NameError: name 'granules' is not defined

In [21]:
API_request

'https://n5eil02u.ecs.nsidc.org/egi/request?short_name=ATL03&version=001&bounding_box=128.3700,27.0000,128.4848,27.0900&page_size=10&token=B066800C-41EE-2F69-CA97-E9D7176B29FC&request_mode=async'

In [30]:
soup = bs4.BeautifulSoup(r.text, 'xml')
soup


<?xml version="1.0" encoding="utf-8"?>
<eesi:agentResponse xmlns="" xmlns:eesi="http://eosdis.nasa.gov/esi/rsp/e" xmlns:esi="http://eosdis.nasa.gov/esi/rsp" xmlns:iesi="http://eosdis.nasa.gov/esi/rsp/i" xmlns:ssw="http://newsroom.gsfc.nasa.gov/esi/rsp/ssw" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://eosdis.nasa.gov/esi/rsp/e https://newsroom.gsfc.nasa.gov/esi/8.1/schemas/ESIAgentResponseExternal.xsd">
<downloadUrls>
<downloadUrl>ftp://n5eil02u.ecs.nasa.gov/DP7/ATLAS/ATL03.001/2018.10.21/ATL03_20181021062956_03460101_001_01.h5</downloadUrl>
<downloadUrl>ftp://n5eil02u.ecs.nasa.gov/DP7/ATLAS/ATL03.001/2018.10.21/ATL03_20181021063658_03460102_001_01.h5</downloadUrl>
<downloadUrl>ftp://n5eil02u.ecs.nasa.gov/DP7/ATLAS/ATL03.001/2018.12.04/ATL03_20181204162952_10240106_001_01.h5</downloadUrl>
<downloadUrl>ftp://n5eil02u.ecs.nasa.gov/DP7/ATLAS/ATL03.001/2018.12.04/ATL03_20181204163822_10240107_001_01.h5</downloadUrl>
<downloadUrl>ftp://n5eil02u.ecs.nasa.go

https://n5eil02u.ecs.nsidc.org/egi/request?short_name=ATL03&version=001&page_size=99&token=C1D565F3-D5F4-4109-FF1B-757BEC07B887&bbox=128.3700,27.0000,128.4848,27.0900&bounding_box=128.3700,27.0000,128.4848,27.0900&request_mode=async

In [61]:
path = str(os.getcwd() + '/Data/Downloaded')
if not os.path.exists(path):
    os.mkdir(path)

In [83]:
session.get(urls[0])

InvalidSchema: No connection adapters were found for 'ftp://n5eil02u.ecs.nasa.gov/DP7/ATLAS/ATL03.001/2018.10.21/ATL03_20181021062956_03460101_001_01.h5'

In [87]:
# Request data service for each page number, and unzip outputs

for i in range(page_num):
    page_val = i + 1
    print('Order: ', page_val)
    params.update( {'page_num': page_val} )
    
# For all requests other than spatial file upload, use get function
    request = session.get(base_url, params=params)
    
    print('Request HTTP response: ', request.status_code)

# Raise bad request: Loop will stop for bad response code.
    request.raise_for_status()
    print('Order request URL: ', request.url)
    esir_root = ET.fromstring(request.content)
    print()
    print('Order request response XML content: ', request.content)
    print()
    
#Look up order ID
    orderlist = []   
    for order in esir_root.findall("./order/"):
        orderlist.append(order.text)
        
    print(orderlist)
    orderID = orderlist[0]
    print('order ID: ', orderID)

#Create status URL
    statusURL = base_url + '/' + orderID
    print('status URL: ', statusURL)

#Find order status
    request_response = session.get(statusURL)    
    print('HTTP response from order response URL: ', request_response.status_code)
    
# Raise bad request: Loop will stop for bad response code.
    request_response.raise_for_status()
    request_root = ET.fromstring(request_response.content)
    statuslist = []
    for status in request_root.findall("./requestStatus/"):
        statuslist.append(status.text)
    status = statuslist[0]
    print('Data request ', page_val, ' is submitting...')
    print('Initial request status is ', status)

#Continue loop while request is still processing
    while status == 'pending' or status == 'processing': 
        print('Status is not complete. Trying again.')
        time.sleep(10)
        loop_response = session.get(statusURL)

# Raise bad request: Loop will stop for bad response code.
        loop_response.raise_for_status()
        loop_root = ET.fromstring(loop_response.content)

#find status
        statuslist = []
        for status in loop_root.findall("./requestStatus/"):
            statuslist.append(status.text)
        status = statuslist[0]
        print('Retry request status is: ', status)
        if status == 'pending' or status == 'processing':
            continue

#Order can either complete, complete_with_errors, or fail:
# Provide complete_with_errors error message:
    if status == 'complete_with_errors' or status == 'failed':
        messagelist = []
        for message in loop_root.findall("./processInfo/"):
            messagelist.append(message.text)
        print('error messages:')
        pprint.pprint(messagelist)

# Download zipped order if status is complete or complete_with_errors
    if status == 'complete' or status == 'complete_with_errors':
        downloadURL = 'https://n5eil02u.ecs.nsidc.org/esir/' + orderID + '.zip'
        print('Zip download URL: ', downloadURL)
        print('Beginning download of zipped output...')
        zip_response = session.get(downloadURL)
        # Raise bad request: Loop will stop for bad response code.
        zip_response.raise_for_status()
        with zipfile.ZipFile(io.BytesIO(zip_response.content)) as z:
            z.extractall(path)
        print('Data request', page_val, 'is complete.')
    else: print('Request failed.')

In [88]:
request.content

NameError: name 'request' is not defined

In [89]:

base_url

'https://n5eil02u.ecs.nsidc.org/egi/request'